In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv('Data/df_clean.csv',index_col="time")

y_price = df['price actual']

# Split the data based on the specific date
def split_by_date(X,train_end_date):
    if not isinstance(train_end_date, pd.Timestamp):
        train_end_date = pd.Timestamp(train_end_date)

    # Convert index of X and y to Timestamp objects if they are strings
    if isinstance(X.index[0], str):
        X.index = pd.to_datetime(X.index)
        
    X_train = X[X.index <= train_end_date]
    X_test = X[X.index > train_end_date]
    
    return X_train, X_test

train_end_date = pd.to_datetime('2017-12-31 23:00:00+00:00')

# X_train, X_test also contains y variable (price actual)
X_train, X_test = split_by_date(df,train_end_date)
y_price_train, y_price_test = split_by_date(y_price, train_end_date)

In [84]:
def split_array(X_arr, y_arr, n_hours):
    X, y = list(), list()
    for window_start in range(len(X_arr)):
        past_end = window_start + n_hours
        if past_end + 1 > len(X_arr):
            break
        past, future = X_arr[window_start:past_end], y_arr[past_end]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)
        
        

In [86]:
n_hours = 3 # Predict the next 3 hours 
X_train_arr, X_test_arr = X_train.values, X_test.values 
y_price_train_arr, y_price_test_arr = y_price_train.values, y_price_test.values


In [87]:
X_train_split, y_train_split =split_array(X_train_arr, y_price_train_arr,n_hours)
test_train24_combine_X = np.concatenate([X_train_arr[-n_past:],X_test_arr])
test_train24_combine_y = np.concatenate([y_price_train_arr[-n_past:],y_price_test_arr])
X_test_split, y_test_split = split_array(test_train24_combine_X, test_train24_combine_y ,n_hours)

In [94]:
X_train['price actual']

time
2014-12-31 23:00:00+00:00    65.41
2015-01-01 00:00:00+00:00    64.92
2015-01-01 01:00:00+00:00    64.48
2015-01-01 02:00:00+00:00    59.32
2015-01-01 03:00:00+00:00    56.04
                             ...  
2017-12-31 19:00:00+00:00    39.54
2017-12-31 20:00:00+00:00    32.90
2017-12-31 21:00:00+00:00    23.85
2017-12-31 22:00:00+00:00    23.95
2017-12-31 23:00:00+00:00    20.76
Name: price actual, Length: 26305, dtype: float64

In [90]:
X_train_split[0][:,19] # Three hours previous sequence

array([65.41, 64.92, 64.48])

In [89]:
y_train_split # You can see that the first value is the next three hour value 

array([59.32, 56.04, 53.63, ..., 23.85, 23.95, 20.76])